<a href="https://colab.research.google.com/github/rybott/FEFF_War_Games/blob/main/Company_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Modules Used

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import plotly.express as px
import random
import duckdb as ddb
import matplotlib.pyplot as plt


# Generating the Sales Data for the Company

In [ ]:
def generate_sales_data(num_records, start_date, end_date, regions_df, products_df):
    # Lists to store generated data
    order_dates = []
    shipment_dates = []
    regions = []
    countries = []
    product_types = []
    unit_prices = []
    unit_costs = []
    quantities = []

    # Generate data for each record
    for _ in range(num_records):
        # Randomly select region and corresponding country
        region_row = regions_df.sample().iloc[0]
        regions.append(region_row['StoreID'])
        countries.append(region_row['StoreName'])

        # Randomly select product and corresponding unit price and cost
        product_row = products_df.sample().iloc[0]
        product_types.append(product_row['Product'])
        unit_prices.append(product_row['Unit Price'])
        unit_costs.append(product_row['Unit Cost'])

        # Random order date between start and end date
        random_date = start_date + timedelta(days=np.random.randint(0, (end_date-start_date).days))
        order_dates.append(random_date)
        # Ship date between 1 and 50 days after order date
        shipment_dates.append(random_date + timedelta(days=np.random.randint(1, 51)))

        # Random quantity between 1000 and 10000
        quantities.append(np.random.randint(1000, 10001))

    # Create dataframe
    sales_df = pd.DataFrame({
        'Order_Date': order_dates,
        'Shipment_Date': shipment_dates,
        'StoreID': regions,
        'StoreName': countries,
        'Product_Type': product_types,
        'Unit_Price': unit_prices,
        'Unit_Cost': unit_costs,
        'Quantity': quantities,
        'Revenue': np.array(unit_prices) * np.array(quantities),
        'Total_Cost': np.array(unit_costs) * np.array(quantities)
    })

    sales_df['Total Profit'] = sales_df['Revenue'] - sales_df['Total_Cost']

    return sales_df

# Example usage:
regions_data = {
    'StoreID': [1,2,3,4,5,6,7,8,9,10],
    'StoreName': ["StoreNY","StoreLA","StorePA","StoreKY","StoreWA","StoreDC","StoreFL","StoreAL","StoreTX","StoreMN",]
}
regions_df = pd.DataFrame(regions_data)

products_data = {
    'Product': ["Office Supplies","Vegetables","Fruits","Cosmetics","Cereal","Baby Food","Beverages","Snacks","Clothes","Household","Personal Care","Meat"],
    'Unit Price': [651.21,154.06,9.33,437.2,205.7,255.28,47.45,152.58,109.28,668.27,81.73,421.89],
    'Unit Cost': [524.96,90.93,6.92,263.33,117.11,159.42,31.79,97.44,35.84,502.54,56.67,364.69]
}
products_df = pd.DataFrame(products_data)

start_date = pd.Timestamp('2020-01-01')
end_date = pd.Timestamp('2022-01-01')
num_records = 1000

sales_df = generate_sales_data(num_records, start_date, end_date, regions_df, products_df)
print(sales_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order_Date     1000 non-null   datetime64[ns]
 1   Shipment_Date  1000 non-null   datetime64[ns]
 2   StoreID        1000 non-null   int64         
 3   StoreName      1000 non-null   object        
 4   Product_Type   1000 non-null   object        
 5   Unit_Price     1000 non-null   float64       
 6   Unit_Cost      1000 non-null   float64       
 7   Quantity       1000 non-null   int64         
 8   Revenue        1000 non-null   float64       
 9   Total_Cost     1000 non-null   float64       
 10  Total Profit   1000 non-null   float64       
dtypes: datetime64[ns](2), float64(5), int64(2), object(2)
memory usage: 86.1+ KB
None


# Create Purchase Order Data

In [ ]:
# df = pd.read_excel('Sales_Data_500k.xlsx')
data = sales_df.copy()

df = data.copy()
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order_Date     1000 non-null   datetime64[ns]
 1   Shipment_Date  1000 non-null   datetime64[ns]
 2   StoreID        1000 non-null   int64         
 3   StoreName      1000 non-null   object        
 4   Product_Type   1000 non-null   object        
 5   Unit_Price     1000 non-null   float64       
 6   Unit_Cost      1000 non-null   float64       
 7   Quantity       1000 non-null   int64         
 8   Revenue        1000 non-null   float64       
 9   Total_Cost     1000 non-null   float64       
 10  Total Profit   1000 non-null   float64       
dtypes: datetime64[ns](2), float64(5), int64(2), object(2)
memory usage: 86.1+ KB
None


### Data Analysis #1
Checking for any errors or irregulairities
I am also Learning SQL Currently

In [ ]:
qry1 = '''
SELECT DISTINCT Product_Name
FROM df
'''

qry2='''
SELECT MAX(Order_Date),MIN(Order_Date)
FROM df
'''

qry4 = '''
SELECT Quantity, Count(Quantity) as sold
FROM df
GROUP BY Quantity
ORDER BY Quantity
'''

qry3 = '''
SELECT Revenue, Count(Revenue) as Rev
FROM df
GROUP BY Revenue
ORDER BY Revenue
'''

df3 = ddb.sql(qry4).df()
print(df3)

In [ ]:
plt.plot(df3.index, df3["sold"])
plt.show()

## Creating the Purchase Order

In [ ]:
# Load your sales data (assuming it's in a CSV file)
sales_data = df
sales_data = sales_data.sort_values(by=['Order_Date'])


# Define product types
product_types = ['Cereal', 'Snacks', 'Beverages', 'Baby Food', 'Meat', 'Fruits', 'Vegetables', 'Personal Care', 'Cosmetics', 'Household', 'Office Supplies', 'Clothes']

# Initialize empty dataframes for purchase orders and inventory
purchase_orders = pd.DataFrame(columns=['OrderID', 'Product', 'Vendor', 'Quantity', 'OrderDate'])
Inventory_dict = {'Cereal':0, 'Snacks':0, 'Beverages':0, 'Baby Food':0, 'Meat':0, 'Fruits':0, 'Vegetables':0, 'Personal Care':0, 'Cosmetics':0, 'Household':0, 'Office Supplies':0, 'Clothes':0}
Vendor_dict = {'Cereal':'Foodco', 'Snacks':'Foodco', 'Beverages':'Foodco', 'Baby Food':'Foodco', 'Meat':'Farmco', 'Fruits':'Farmco', 'Vegetables':'Farmco', 'Personal Care':'Beautyco', 'Cosmetics':'Beautyco', 'Household':'Homeco', 'Office Supplies':'Homeco', 'Clothes':'Fashionco'}

# Purchase Order lists
Product_Ordered = []
Quantity_Purchased = []
Date_Purchased = []
Vendor = []


# Define a function to generate purchase orders

for index, row in sales_data.iterrows():
  x = 0
  product = row['Product_Type']
  quantity_sold = row['Quantity']
  order_date = row['Order_Date']
  Unit_Cost = row['Unit_Cost']

  if product in Inventory_dict.keys():
   if quantity_sold > Inventory_dict[product]:
    purchase_quantity = (quantity_sold - Inventory_dict[product]) * 1.05
    Inventory_dict[product] = (purchase_quantity + Inventory_dict[product]) - quantity_sold
    Product_Ordered.append(product)
    Quantity_Purchased.append(purchase_quantity)
    Date_Purchased.append(order_date)
    Ven = Vendor_dict[product]
    Vendor.append(Ven)
  else:
    Inventory_dict[product] = Inventory_dict[product] - quantity_sold

PO_df = pd.DataFrame(
    {'Vendor': Vendor,
     'Quantity': Quantity_Purchased,
     'Product': Product_Ordered,
     'Order_Date': Date_Purchased,
     'Unit_Cost': Unit_Cost
    })

Qry_PO ='''
    SELECT Vendor,
    Product,
    SUM(Quantity),
    YEAR(Order_Date) AS Year,
    QUARTER(Order_Date) AS Quarter,
    CAST(
      CASE
        WHEN QUARTER(Order_Date) = 1
          THEN CONCAT(YEAR(Order_Date), '-01-01')
        WHEN QUARTER(Order_Date) = 2
          THEN CONCAT(YEAR(Order_Date), '-04-01')
        WHEN QUARTER(Order_Date) = 3
          THEN CONCAT(YEAR(Order_Date), '-07-01')
        WHEN QUARTER(Order_Date) = 4
          THEN CONCAT(YEAR(Order_Date), '-10-01')
      END AS DATE) AS Purchase_Date
    FROM PO_df
    GROUP BY Vendor, Product, YEAR(Order_Date), QUARTER(Order_Date)
    ORDER BY Year, Quarter
'''

df_PO = ddb.sql(Qry_PO).df()
print(df_PO.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Vendor         96 non-null     object        
 1   Product        96 non-null     object        
 2   sum(Quantity)  96 non-null     float64       
 3   Year           96 non-null     int64         
 4   Quarter        96 non-null     int64         
 5   Purchase_Date  96 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 4.6+ KB
None


In [ ]:
with pd.ExcelWriter('POs.xlsx') as writer:
    df_PO.to_excel(writer)

# Creating Inventory
First Attempt a Failure

In [ ]:
Inv_DEC = pd.DataFrame({"order_date" : df['Order_Date'],"Product":df["Product_Type"],"Quantity":df["Quantity"],"Unit_Cost":df["Unit_Cost"]})
Inv_INC = pd.DataFrame({"Purchase_Date": df_PO['Purchase_Date'],"Product":df_PO['Product'],"Quantity":df_PO['sum(Quantity)']})

qry_INV = '''
SELECT Inv_DEC.order_date, Inv_DEC.Purchase_Date, Inv_DEC.Product, Inv_DEC.Quantity, Inv_INC.Quantity, Inv_DEC.Unit_Cost
FROM Inv_DEC
FULL OUTER JOIN Inv_INC ON Inv_DEC.order_date = Inc_INC.Purchase_Date
ORDER BY order_date
'''

df_INV = ddb.sql(qry_INV).df()
print(df_INV.info())

In [ ]:
%%shell
jupyter nbconvert --to html /content/Company_Generator.ipynb

[NbConvertApp] Converting notebook /content/Company_Generator.ipynb to html
[NbConvertApp] Writing 614871 bytes to /content/Company_Generator.html
